In [1]:
import re
import numpy as np
import pandas as pd 
from konlpy.tag import Okt
import pyLDAvis
import pyLDAvis.gensim
from gensim import corpora 
from gensim import models
import gensim

C:\Users\yj658\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
def trans(rowdata,kingname):
    rmhdata = []
    for num1 in range(len(rowdata)):
        for num in range(len(rowdata[num1])):
            rmhdata.append(re.sub('[(一-龥)]','',rowdata[num1][num]))
    wdata =[]
    for i in range(len(rmhdata)):
        wdata.append(re.sub('-|○|/|。|=|\n','',rmhdata[i]))
    
    sdata =[]
    for i in range(len(wdata)):
        if wdata[i] != '':
            sdata.append(wdata[i])
    ssdata =[]
    for i in range(len(sdata)):
        if re.search('[가-힣]+',sdata[i]):
            ssdata.append(sdata[i])
    data =[]
    for i in range(len(ssdata)):
        if kingname not in ssdata[i]:
           data.append(ssdata[i])

    return data

In [3]:
data =[]
for i in range(0,1):
    txt1 = open('고종.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '고종' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '백성' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'고종'))

In [5]:
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("고종실록"," "))
df=df.map(lambda x : x.replace("고종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))
len(df)

2887

In [6]:
okt = Okt()

In [7]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금','어찌','모두','우리'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [8]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

In [9]:
 # 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary


In [10]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	백성	0.02149043045938015
	마음	0.011818520724773407
	하늘	0.007955018430948257
	폐하	0.006072536110877991
	대해	0.005951710976660252
	생각	0.0055123805068433285
	하라	0.005015685223042965
	하니	0.00481912586838007
	소자	0.004507262725383043
	조선	0.0038453617598861456


Topic ID: 1
	백성	0.01138575840741396
	하니	0.008280650712549686
	협상	0.006174864247441292
	장호	0.005547888111323118
	시행	0.00543301971629262
	대한	0.004953100811690092
	개국	0.0046774717047810555
	하교	0.004608308896422386
	하라	0.0045622773468494415
	전교	0.0043920595198869705


Topic ID: 2
	백성	0.018267467617988586
	하니	0.008614801801741123
	하라	0.007456502411514521
	마음	0.007323142606765032
	조선	0.007046734914183617
	개국	0.006959824822843075
	대한	0.006837956141680479
	대해	0.006665113847702742
	대하	0.004891141317784786
	시행	0.004877905361354351


Topic ID: 3
	백성	0.02681547775864601
	때문	0.007401004433631897
	하니	0.007179583888500929
	생각	0.007154026534408331
	폐단	0.006095609627664089
	마음	0.005972295068204403
	하라	0.0057955938391387

In [11]:
# 주제 :  백성
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12     0.080598  0.064363       1        1  17.296492
17    -0.052576  0.003182       2        1  13.287562
3      0.000359  0.038419       3        1  10.686529
8     -0.099004  0.001651       4        1   9.815418
11     0.108346 -0.020303       5        1   8.881062
15    -0.024365 -0.028826       6        1   6.390520
2     -0.024742 -0.005594       7        1   5.352938
7     -0.006040  0.043515       8        1   5.140661
14     0.099126 -0.043568       9        1   4.291348
13     0.012259 -0.008666      10        1   4.238542
16    -0.064507  0.020149      11        1   3.276118
4      0.052167 -0.017570      12        1   2.388067
5      0.020607  0.045204      13        1   2.088831
10    -0.023667  0.045483      14        1   1.896343
9      0.004189  0.025811      15        1   1.663459
0      0.035350 -0.019380      16        1   1.609373
6     -0.039272 -0.027922      17        1   0.609405
18    -0.023733 -0.050821      18        1   0.443239
1     -0.029452 -0.037191      19        1   0.331930
19    -0.025643 -0.027937      20        1   0.312161, topic_info=    Term         Freq        Total Category  logprob  loglift
10    백성  7620.000000  7620.000000  Default  30.0000  30.0000
137   하니  4839.000000  4839.000000  Default  29.0000  29.0000
26    조선  1723.000000  1723.000000  Default  28.0000  28.0000
156   마음  2734.000000  2734.000000  Default  27.0000  27.0000
87    생각  3018.000000  3018.000000  Default  26.0000  26.0000
..   ...          ...          ...      ...      ...      ...
30    폐단     3.325445  1265.035034  Topic20  -6.0826  -0.1718
208   시행     3.583842  2019.636108  Topic20  -6.0078  -0.5648
771   경우     3.211689  1055.045044  Topic20  -6.1174  -0.0251
822   다시     3.165509  1537.740479  Topic20  -6.1319  -0.4163
402   비록     2.888752   987.895386  Topic20  -6.2234  -0.0653

[1595 rows x 6 columns], token_table=       Topic      Freq Term
term                       
5732       1  0.036607   가격
5732       2  0.122023   가격
5732       3  0.024405   가격
5732       4  0.561304   가격
5732       5  0.012202   가격
...      ...       ...  ...
2217      17  0.004596   흉년
2217      18  0.006895   흉년
2217      19  0.002298   흉년
2217      20  0.002298   흉년
13169      6  0.701386  흑단령

[6933 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 18, 4, 9, 12, 16, 3, 8, 15, 14, 17, 5, 6, 11, 10, 1, 7, 19, 2, 20])

In [ ]:
# 개국

In [24]:
data =[]
for i in range(0,1):
    txt1 = open('고종.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '고종' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '명' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'고종'))

In [25]:
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("고종실록"," "))
df=df.map(lambda x : x.replace("고종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))
len(df)

13371

In [26]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금','어찌','모두','우리','하니'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

 # 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary

In [27]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	추증	0.01607700064778328
	제사	0.013481161557137966
	박용대	0.01334497332572937
	은전	0.013107610866427422
	아들	0.012824634090065956
	절개	0.011463535949587822
	상소	0.010713160037994385
	시호	0.010163050144910812
	충성	0.009304377250373363
	정공	0.008646184578537941


Topic ID: 1
	죄인	0.02033095434308052
	역적	0.016390936449170113
	군사	0.010995842516422272
	무리	0.008926033973693848
	대해	0.008225543424487114
	해당	0.00799919106066227
	대한	0.007791557814925909
	대군	0.007142658345401287
	적용	0.007081137504428625
	보고	0.007065445184707642


Topic ID: 2
	임명	0.0654388889670372
	개국	0.04818105325102806
	조선	0.046288181096315384
	이조	0.034440163522958755
	임용	0.02536201663315296
	대한	0.02504812553524971
	제조	0.025038395076990128
	판서	0.021054914221167564
	관찰사	0.01991353929042816
	정품	0.017611978575587273


Topic ID: 3
	개국	0.07669162005186081
	조선	0.07454763352870941
	임명	0.07304373383522034
	의금부	0.0455302819609642
	한성부	0.02823617123067379
	판윤	0.026687031611800194
	수군절도사	0.022972790524363518
	병마절도사

In [28]:
# 주제 :  개국
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.169775  0.108581       1        1  20.256239
8     -0.103250 -0.014890       2        1  18.704500
19    -0.156901  0.160327       3        1   8.889183
14    -0.171038  0.010790       4        1   7.371703
2      0.280105  0.002301       5        1   5.837273
1     -0.121423 -0.022295       6        1   5.425250
7      0.008697 -0.099901       7        1   4.570802
5     -0.035485 -0.277281       8        1   3.857765
15     0.011371  0.035456       9        1   3.137521
17     0.031247  0.033029      10        1   3.110276
12    -0.036618 -0.244858      11        1   2.961487
4     -0.043922  0.087285      12        1   2.590255
13     0.208757 -0.008400      13        1   2.398202
3      0.278824  0.060863      14        1   2.049439
6     -0.035002  0.050240      15        1   1.877783
9     -0.074089 -0.054949      16        1   1.707360
0     -0.076560  0.100477      17        1   1.654673
16    -0.021057 -0.000690      18        1   1.423314
11     0.026662 -0.002396      19        1   1.387565
18     0.199458  0.076311      20        1   0.789418, topic_info=     Term         Freq        Total Category  logprob  loglift
25     임명  6941.000000  6941.000000  Default  30.0000  30.0000
2959   임용  4047.000000  4047.000000  Default  29.0000  29.0000
1      개국  7759.000000  7759.000000  Default  28.0000  28.0000
28     조선  8078.000000  8078.000000  Default  27.0000  27.0000
81     대신  6590.000000  6590.000000  Default  26.0000  26.0000
...   ...          ...          ...      ...      ...      ...
788    임시    44.523968  1084.136963  Topic20  -4.9858   1.6491
1649   재상    28.815842   230.212738  Topic20  -5.4209   2.7636
490   의금부    36.668789  1149.990112  Topic20  -5.1799   1.3960
1941  영의정    32.662235   830.408203  Topic20  -5.2956   1.6059
142    제수    26.333912   695.586792  Topic20  -5.5110   1.5677

[1409 rows x 6 columns], token_table=      Topic      Freq Term
term                      
7245      1  0.007129   가격
7245      2  0.057033   가격
7245      4  0.007129   가격
7245      8  0.805589   가격
7245     11  0.071291   가격
...     ...       ...  ...
3313     16  0.897042   흉배
2387      3  0.957266   흉역
6585      1  0.942511   흥망
9411     19  0.945753   흥양
1589      2  0.955590  흥인군

[7318 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 9, 20, 15, 3, 2, 8, 6, 16, 18, 13, 5, 14, 4, 7, 10, 1, 17, 12, 19])